In [1]:
import torch
import os
import pymupdf
import transformers

In [2]:
pdf_path = "resume.pdf"
doc = pymupdf.open(pdf_path)


In [3]:
for page in doc:
    text = page.get_text()
    text = text.strip()
    print(text)

Senior Software Engineer at Infosys Technologies Aug 2019 –PresentIndianapolis, IN
 Successfully developed methodologies to streamline the process of 
data conversion leveraging latest tools
 Created optimized PL/SQL scripts to automate data gathering and 
transformation on various database technologies
 Refactored existing processes and built component to reduce 
manual efforts and performed unit tests
 Conducted meetings and workshops with client to document 
requirements and provide roadmap for the project
Java Developer at School of Education Jan –May 2019College Park, MD
 Successfully architectured and implemented features in Spring 
Boot to enhance the capabilities of  web application
 Designed and developed various UI pages in ReactJS to support 
application and contributed in creating process workflows
 Achieved a maintainable and optimised code by refactoring and 
following best coding and design practices
 Implemented unit tests in JUnit4 and Mockito providing full
co

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [40]:
from transformers import AutoTokenizer, BitsAndBytesConfig, Gemma3ForCausalLM
import torch

model_id = "google/gemma-3-1b-it"

quantization_config = BitsAndBytesConfig(load_in_8bit=True)

model = Gemma3ForCausalLM.from_pretrained(
    model_id, quantization_config=quantization_config
).eval()

tokenizer = AutoTokenizer.from_pretrained(model_id)

messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are a helpful assistant."}]
    },
    {
        "role": "user",
        "content": [{"type": "text", "text": f"""
        Please summarize the candidate's resume succinctly, noting their work experience, skills, projects and education. 
        in 5 sentences, Begin without any introduction. 
        The answer is to start with a summary, without a comment like Here's a summary. Highlight his experience, education, 
        projects and write a short summary of this cv like examples:

Example 1. The person graduated with a 1st degree in computer science, worked as a java developer from 1/12/2023 to 4/07/2024. 
He is skilled in technologies such as Java, Kotlin, Spring, HTML, CSS. The projects she has done are: 
Web application for checking stock charts, Website design for a game store.

Example 2 A person has completed a bachelor's degree in computer science and has not yet worked. 
She is skilled in technologies such as Python, Pytorch, Numpy, Pandas. The projects she has done are: 
Image classifier, Housing price analysis.

Enter the answer for this text: {text}
        """}]
    },
]

inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,  # Tokenizacja od razu
    return_dict=True,
    return_tensors="pt",  # teraz chcemy zwykły tekst, nie tokeny
).to(model.device)

#inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.inference_mode():
    outputs = model.generate(**inputs, max_new_tokens=256, temperature=0.7)

output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

response_only = output_text.split("model\n")[-1].strip()
print(response_only)


`low_cpu_mem_usage` was None, now default to True since model is quantized.


Here’s a 5-sentence summary of the candidate’s resume:

This Senior Software Engineer brings a strong foundation in Java, Python, and JavaScript, complemented by expertise in database technologies and front-end development.  Their experience includes developing optimized PL/SQL scripts, refactoring processes, and building robust UI components using ReactJS.  The candidate also holds certifications as an Oracle Certified Associate and has demonstrated proficiency in Agile methodologies and API design.  A Master’s degree in Computer Engineering provides a solid theoretical background in computer science and machine learning.  Finally, their background includes a Bachelor’s degree in Electronics and Telecomms Engineering, demonstrating a solid understanding of embedded systems and microcontrollers.


In [41]:
response_only



'Here’s a 5-sentence summary of the candidate’s resume:\n\nThis Senior Software Engineer brings a strong foundation in Java, Python, and JavaScript, complemented by expertise in database technologies and front-end development.  Their experience includes developing optimized PL/SQL scripts, refactoring processes, and building robust UI components using ReactJS.  The candidate also holds certifications as an Oracle Certified Associate and has demonstrated proficiency in Agile methodologies and API design.  A Master’s degree in Computer Engineering provides a solid theoretical background in computer science and machine learning.  Finally, their background includes a Bachelor’s degree in Electronics and Telecomms Engineering, demonstrating a solid understanding of embedded systems and microcontrollers.'

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
df = pd.read_csv("data/UpdatedResumeDataSet.csv")
df.head()

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 962 entries, 0 to 961
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  962 non-null    object
 1   Resume    962 non-null    object
dtypes: object(2)
memory usage: 15.2+ KB


In [8]:
df.describe()

,Category,Resume
count,962,962
unique,25,166
top,Java Developer,"Technical Skills Web Technologies: Angular JS,..."
freq,84,18


In [9]:
df.isnull().sum()

Category    0
Resume      0
dtype: int64

In [10]:
df.isna().sum()

Category    0
Resume      0
dtype: int64

In [11]:
df["Resume"] = df["Resume"].str.replace("\r\n", "")


In [12]:
df["Resume"]

0      Skills * Programming Languages: Python (pandas...
1      Education Details May 2013 to May 2017 B.E   U...
2      Areas of Interest Deep Learning, Control Syste...
3      Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4      Education Details  MCA   YMCAUST,  Faridabad, ...
                             ...                        
957    Computer Skills: â¢ Proficient in MS office (...
958    â Willingness to accept the challenges. â ...
959    PERSONAL SKILLS â¢ Quick learner, â¢ Eagerne...
960    COMPUTER SKILLS & SOFTWARE KNOWLEDGE MS-Power ...
961    Skill Set OS Windows XP/7/8/8.1/10 Database MY...
Name: Resume, Length: 962, dtype: object

In [28]:
df["Resume"][1]

'Education Details May 2013 to May 2017 B.E   UIT-RGPVData Scientist Data Scientist - MatelabsSkill Details Python- Exprience - Less than 1 year monthsStatsmodels- Exprience - 12 monthsAWS- Exprience - Less than 1 year monthsMachine learning- Exprience - Less than 1 year monthsSklearn- Exprience - Less than 1 year monthsScipy- Exprience - Less than 1 year monthsKeras- Exprience - Less than 1 year monthsCompany Details company - Matelabsdescription - ML Platform for business professionals, dummies and enthusiasts.60/A Koramangala 5th block,Achievements/Tasks behind sukh sagar, Bengaluru,India                               Developed and deployed auto preprocessing steps of machine learning mainly missing valuetreatment, outlier detection, encoding, scaling, feature selection and dimensionality reduction.Deployed automated classification and regression model.linkedin.com/in/aditya-rathore-b4600b146                           Reasearch and deployed the time series forecasting model ARIMA, S